# My Notes 

## Research papers of interest
These papers are stored at the Arxiv website, they are mostly have not been through peer reviews because they are so new. On the other hand, they are follow up and performed by others researchers and interested users on the internet, if there are problems these will be report almost instantly on the social network.

[diffedit](https://arxiv.org/abs/2210.11427)  
[Progressive Distillation for Fast Sampling of Diffusion Models](https://arxiv.org/abs/2202.00512)  
[On Distillation of Guided Diffusion Models](https://arxiv.org/abs/2210.03142)  
[Imagic: Text-Based Real Image Editing with Diffusion Models](https://arxiv.org/abs/2210.09276)  



## Eistein Summation

A technique for convenient and simplify the writing of syntax for many operation in linear algebra

- Matrix multiplication
- Element-wise matrix operation
- Permutation of matrix
- Dot product of matrix
- Outer product of matrix
- Summation of matrix
- Batch multiplication of matrix (permute input to match function calls ordering)

It's also speedup some of the above operation especially operation that can be combined into single call

The eisum is a build-in feature of most Machine learning frameworks, e.g. Pytorch, Tensorflow, Flux...

How does it work?

Here is the matrix multiplication  
$$ M_{ij} = \sum{A_{ik}B_{kj}} = A_{ik}B_{kj}$$

In [1]:
import numpy as np

In [2]:
A = np.random.rand(3,5)
B = np.random.rand(5,2)
M = np.empty((3,2))

A.shape,B.shape,M.shape

((3, 5), (5, 2), (3, 2))

In [3]:
A,B,M

(array([[6.82759644e-01, 3.32325362e-01, 5.10214353e-01, 8.76891766e-01,
         1.72299180e-01],
        [5.33438050e-01, 7.25523710e-01, 8.54886776e-01, 3.96140734e-01,
         3.56604238e-01],
        [5.45349948e-05, 8.97919640e-01, 1.86133084e-01, 8.73382052e-01,
         1.11479305e-01]]),
 array([[0.18449366, 0.69112411],
        [0.68456084, 0.92427429],
        [0.39745901, 0.59819685],
        [0.53124429, 0.82624474],
        [0.37620928, 0.55749668]]),
 array([[1.13621302e-313, 0.00000000e+000],
        [6.92770534e-310, 6.92770729e-310],
        [6.92770292e-310, 6.92770291e-310]]))

In [4]:
np.set_printoptions(precision=2, linewidth=140)

### Code:  
Coding as loop of the matrix multiplication above   
where the row is i and column is j, and k is the inner dimension of both matrix that must be equal and this index will be summed and disappeared by the operation of matrix rule

In [5]:
for i in range(3):
    for j in range(2):
        total = 0
        for k in range(5):
            total += A[i,k] * B[k,j]
        M[i,j] = total

print(f'the matrix is: {M}')

the matrix is: [[1.09 1.9 ]
 [1.28 2.08]
 [1.19 1.73]]


Using the Eisum method

In [6]:

# the i and j are free index
# the k is sum index since it will be summed away after the operation
M1 = np.einsum('ik,kj->ij',A,B)

print(f'the matix is : {M1}')

the matix is : [[1.09 1.9 ]
 [1.28 2.08]
 [1.19 1.73]]


### Dot product
Vector or matrix multiplicaton $u \cdot v$  that result of scalar value  

### Outer product 
vector multiply by vector that result in a matrix $u \otimes v$  

Example 2
- Using the free index in the output  
- No summation index  

In [7]:
# example 2
D = np.random.rand(5)
E = np.random.rand(3)
out = np.einsum('i,j->ij',D,E)
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.32 0.69 0.7 ]
 [0.37 0.8  0.8 ]
 [0.28 0.61 0.61]
 [0.37 0.79 0.79]
 [0.17 0.36 0.36]]


In [8]:
# loop version
for i in range(5):
    for j in range(3):
        total = 0
        total += D[i] * E[j]
        out[i,j] = total
        
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.32 0.69 0.7 ]
 [0.37 0.8  0.8 ]
 [0.28 0.61 0.61]
 [0.37 0.79 0.79]
 [0.17 0.36 0.36]]


## The Eisum Rules

The free indices: 
- The index that specify the output

The Summation index  
- All other indices that appear in the input argument but not show up in the output

The General rules:
1. Same index in the a different input argument indicate that these indices will be multiplied and the product are outputed
``` python
    M = np.einsum('ik,kj->ij',A,B)
```
2. Omitting index indicate the index will be summed together
``` python
    X = np.ones(3)
    Y = np.einsum('i->',X)
```
3. The unsummed indices may return in any order
``` python
    D = np.ones((5,4,3))
    E = np.einsum('ijk->kji',D)
```

Operation that benefit from Einsum
1. Permutation of Tensors
2. Summation
3. Column sum
4. Row sum
5. Matrix-Vector multiplication
6. Matrix-Matrix multiplication
7. Dot Product the first row with first row of a matrix
8. Dot product with matrix (multiplication and add)
9. Element-wise multiplication (Hadamard Product) (multiplication no add)
10. Outer Product
11. Batch matrix multiplicaton e.g. a = 3,2,6 and b = 3,6,3
    - want to multiply the matrix of 2x6 with 6x3 matrix
    - these matrix must follow the multiplication rule
    - the first number  is the batch size they must match, but not count as index 
    - the torch.bmm function will do the same thing
12. Matrix diagonal  
    - return the only the diagonal value of the matrix
13. Matrix Trace
    - summing the value of the diagonal of a matrix  
14. Tensor contration  
    - shrinking the dimension of tensor  
15. Bilinear transformation  

In [9]:
# !pip install torch

In [10]:
import torch
import numpy as np

torch.set_printoptions(precision=2, linewidth=140)

X = torch.rand((2,3))
X

tensor([[0.25, 0.74, 0.66],
        [0.96, 0.80, 0.44]])

1. Transpose
Flipping the matrix or vector by switching the index of a matrix or vector

In [11]:
#| label: mat-transpose
#| code-fold: True

c_ntp = np.transpose(X)
c_tp = torch.transpose(X,0,1)
cein = torch.einsum('ij->ji',X)
print(f'numpy: {c_ntp} \npytorch: {c_tp}\n \neinsum: {cein}')

numpy: tensor([[0.25, 0.96],
        [0.74, 0.80],
        [0.66, 0.44]]) 
pytorch: tensor([[0.25, 0.96],
        [0.74, 0.80],
        [0.66, 0.44]])
 
einsum: tensor([[0.25, 0.96],
        [0.74, 0.80],
        [0.66, 0.44]])


2. Matrix summation  
Summing all value in the matrix that result in a scalar value

In [12]:
#| label: matrix-sum

#2. Summation
cma = torch.sum(X)
cein = torch.einsum('ij->',X)

print(f'regular: {cma} \neinsum: {cein}')

regular: 3.8526394367218018 
einsum: 3.8526394367218018


In [13]:
X

tensor([[0.25, 0.74, 0.66],
        [0.96, 0.80, 0.44]])

Row sum (Left to right)

Add all values from each column together along the row

In [14]:
#| label: row-summation

#4 row summation
# sum by columns
rows = torch.sum(X,dim=0)
cein = torch.einsum('ij->i',X)

print(f'regular: {rows} \neinsum: {cein}')

regular: tensor([1.21, 1.54, 1.10]) 
einsum: tensor([1.65, 2.20])


Column sum (Top down)

Add all value from each row together along the column

In [15]:
#| label: column-summation

#3 Column summation
# sum by rows
c_col = torch.sum(X,dim=1)
cein = torch.einsum('ij->j',X)

print(f'regular: {c_col} \neinsum: {cein}')

regular: tensor([1.65, 2.20]) 
einsum: tensor([1.21, 1.54, 1.10])


5 matrix-vector multiplication  
This a non equal dimension multiplication which in Python use broadcasting to padded (duplicate) the smaller vector to have equal size with the larger matrix before do multiplication

In [16]:
#| label: matrix-vector

#5 matrix-vector multiplication
L = torch.rand((1,3))
M = torch.rand((3,))

cmm = torch.matmul(L,M)
cein = torch.einsum('ij,j->i',L,M)
print(f'regular: {cmm} \neinsum: {cein}')


regular: tensor([0.76]) 
einsum: tensor([0.76])


6 matrix-matrix multiplication  
This standard matrix to matrix multiplication which result in another matrix

In [17]:
#| label: matrix-matrix

#6 matrix-matrix multiplication
# torch.einsum('ij,kj->ik',M,M)

a = torch.ones((3,2))
b = torch.ones((2,3))
cmm = torch.matmul(a,b)
cein = torch.einsum('ij,jl->il',a,b)
print(f'regular: {cmm} \neinsum: {cein}')


regular: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]]) 
einsum: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])


In [18]:
N = torch.rand((3,3))
M = torch.rand((2,3))
torch.einsum('ij,kj->ik',N,M)

tensor([[0.98, 0.72],
        [0.64, 0.60],
        [1.58, 1.42]])

Dot product

This a matrix multiplication that result in a scalar value. It's usually called multiply add.  
since after multiply the row to the column then the sum operation is carry out resulting as a scalar value

In [19]:
#| label: dot-product

# dot product of a matrix (multiply + add)
#torch.einsum('ij,ij->',N,N)

# c = torch.tensor([2,3])
# d = torch.tensor([2,1])
c = torch.rand((3))
d = torch.rand((3))

c_dot = torch.dot(c,d)
cein = torch.einsum('i,i->',c,d)

print(f'c: {c}, c: {c.shape}')
print(f'c_dot: {c_dot}')
print(f'regular: {c_dot} \n  einsum: {cein}')

c: tensor([0.38, 0.88, 0.10]), c: torch.Size([3])
c_dot: 0.3021569550037384
regular: 0.3021569550037384 
  einsum: 0.3021569550037384


In [20]:
# dot product of only the first row of a matrix with first row of a matrix
torch.einsum('i,i->',N[0],N[0])

tensor(0.89)

Hadamard Product 
Element wise multiplication (multiply only)  
This is a normal matrix multiplication which different from multiply add or dot product

In [21]:
#| label: Hadamard-product

# element wise multiplication (multiply only)
# torch.einsum('ij,ij->ij',N,N)

c = torch.randn((3,2))
d = torch.randn((3,2))
cmm = c * d
cein = torch.einsum('ij,ij->ij',c,d)
print(f'regular: {cmm} \n  einsum: {cein}')

regular: tensor([[ 1.20,  0.97],
        [ 0.14, -0.19],
        [-0.17, -0.11]]) 
  einsum: tensor([[ 1.20,  0.97],
        [ 0.14, -0.19],
        [-0.17, -0.11]])


Outer Product

Multiply vector of different size to get a matrix as output
In eisum must use different letter for index to represent size different


In [22]:
#| label: outer-product

# outer product (inner product)
x = torch.rand(3)
y = torch.rand(5)
print(f'x: {x}, x: {x.shape}')
print(f'y: {y}, y: {y.shape}')

c_outer = torch.outer(x,y)
cein = torch.einsum('i,j->ij',x,y)
print(f'regular: {c_outer} \n  einsum: {cein}')

x: tensor([0.91, 0.93, 0.21]), x: torch.Size([3])
y: tensor([0.98, 0.62, 0.25, 0.90, 0.21]), y: torch.Size([5])
regular: tensor([[0.89, 0.56, 0.23, 0.81, 0.19],
        [0.91, 0.57, 0.23, 0.83, 0.19],
        [0.20, 0.13, 0.05, 0.18, 0.04]]) 
  einsum: tensor([[0.89, 0.56, 0.23, 0.81, 0.19],
        [0.91, 0.57, 0.23, 0.83, 0.19],
        [0.20, 0.13, 0.05, 0.18, 0.04]])


Batch matrix multiplication  
Multiply matrix by the set of n, where n is batch size  

want to multiply 3 set of the matrix of 2x6 with 6x3 matrix  
the first number is the batch size must match but not count as index so i is ignore  
the mxn * nxp must match with n

In [23]:
#| label: batch-matrix

# batch matrix multiplicaton
# want to multiply 3 set of the matrix of 2x6 with 6x3 matrix
# the first number is the batch size must match but not count as index so i is ignore
# the mxn * nxp must match with n
R = torch.rand(3,2,6)
S = torch.rand(3,6,3)
cmn = np.matmul(R,S)
cmm = torch.matmul(R,S)

cein = torch.einsum('ijk,ikl->ijl',R,S)

print(f'regular: {cmm}\n numpy: {cmn} \n  einsum: {cein}')

regular: tensor([[[1.55, 1.48, 1.68],
         [1.26, 1.02, 1.42]],

        [[1.81, 1.20, 1.37],
         [1.91, 0.71, 0.93]],

        [[0.75, 0.92, 1.20],
         [1.32, 1.29, 1.85]]])
 numpy: tensor([[[1.55, 1.48, 1.68],
         [1.26, 1.02, 1.42]],

        [[1.81, 1.20, 1.37],
         [1.91, 0.71, 0.93]],

        [[0.75, 0.92, 1.20],
         [1.32, 1.29, 1.85]]]) 
  einsum: tensor([[[1.55, 1.48, 1.68],
         [1.26, 1.02, 1.42]],

        [[1.81, 1.20, 1.37],
         [1.91, 0.71, 0.93]],

        [[0.75, 0.92, 1.20],
         [1.32, 1.29, 1.85]]])


Diagonal Matrix
return the vector of value along the diagonal of a matrix

In [24]:
#| label: diag-mat

# Diagonal matrix return only the diagonal value of a matrix

T = torch.rand(3,3)

cein = torch.einsum('ii->i',T)
print(f'T: {T} \nT shape: {T.shape}')
c_diag = torch.diag(T)

print(f'regular: {c_diag} \n  einsum: {cein}')


T: tensor([[0.33, 0.11, 0.63],
        [0.60, 0.82, 0.28],
        [0.08, 0.99, 0.18]]) 
T shape: torch.Size([3, 3])
regular: tensor([0.33, 0.82, 0.18]) 
  einsum: tensor([0.33, 0.82, 0.18])


In [25]:
# torch.einsum('ii->i',T)

Trace

Take the sum of all values along the diagonal axix of a matrix

In [26]:
#| label: trace

# matrix trace

c_trace = torch.trace(T)
cein = torch.einsum('ii->',T)
print(f'T: {T}')
print(f'regular: {c_trace} \n  einsum: {cein}')

T: tensor([[0.33, 0.11, 0.63],
        [0.60, 0.82, 0.28],
        [0.08, 0.99, 0.18]])
regular: 1.3291736841201782 
  einsum: 1.3291736841201782


Tensor Contraction  
Shrinking the dimension of the tensor  
must provide the dimension to be ignored


In [27]:
#| label: contraction

o = torch.rand((3,4,2))
p = torch.rand((4,3,6))
print(f'value: {o.shape} value2: {p.shape}')

c_tdot = torch.tensordot(o,p,dims=([1,0],[0,1]))
cein = torch.einsum('ijk,jil->kl',o,p)
print(f'regular: {c_tdot} \n  einsum: {cein}')

value: torch.Size([3, 4, 2]) value2: torch.Size([4, 3, 6])
regular: tensor([[3.72, 2.19, 3.15, 3.26, 2.88, 1.90],
        [5.16, 3.22, 4.84, 4.64, 4.52, 3.33]]) 
  einsum: tensor([[3.72, 2.19, 3.15, 3.26, 2.88, 1.90],
        [5.16, 3.22, 4.84, 4.64, 4.52, 3.33]])


Bilinear transformation

In [28]:
a = torch.rand(2,3)
b = torch.rand(5,3,7)
c = torch.rand(2,7)

torch.einsum('ik,jkl,il->ij',[a,b,c])

tensor([[3.12, 3.11, 2.71, 3.34, 2.55],
        [3.39, 2.72, 2.68, 3.09, 2.96]])